# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [19]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from config import gkey


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [20]:
city_df = pd.read_csv("../output_data/City_Weather_df.csv")
city_df.head()

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,torbay,47.6666,-52.7314,52.68,86,94,10.00,CA,1635131636
1,yellowknife,62.4560,-114.3525,40.41,98,100,17.05,CA,1635131636
2,rikitea,-23.1203,-134.9692,74.68,78,46,16.37,PF,1635131637
3,porto novo,6.4965,2.6036,75.88,87,14,5.64,BJ,1635131638
4,siuna,13.7332,-84.7773,73.15,97,43,2.35,NI,1635131639


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [21]:
locations = city_df[["Lat", "Lng"]]
humidity = city_df['Humidity'].astype(int)

# Configure Gmaps
gmaps.configure(api_key=gkey)
fig = gmaps.figure()

# Create heat layer using coordinates and humidity for weighting
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=max(city_df["Humidity"]),
                                 point_radius = 3)

# Add Heatmap layer
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [22]:
# Create new DataFrame fitting weather criteria
Ideal_Cities_df = city_df.copy()

# Narrow down the DataFrame to find your ideal weather condition
# A max temperature lower than 80 degrees but higher than 70.
Ideal_Cities_df = Ideal_Cities_df[Ideal_Cities_df['Max Temp'] >= 70]
Ideal_Cities_df = Ideal_Cities_df[Ideal_Cities_df['Max Temp'] <= 80] 

# Wind speed less than 10 mph.
Ideal_Cities_df = Ideal_Cities_df[Ideal_Cities_df['Wind Speed'] <= 10]

# Zero cloudiness.
Ideal_Cities_df = Ideal_Cities_df[Ideal_Cities_df['Cloudiness'] == 0]

# Drop any rows that don't contain all three conditions. You want to be sure the weather is ideal.
Ideal_Cities_df = Ideal_Cities_df.dropna()
Ideal_Cities_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
12,malangwa,26.8568,85.5581,76.64,71,0,1.43,NP,1635131648
70,tura,25.5198,90.2201,75.38,75,0,2.28,IN,1635131698
167,misratah,32.3754,15.0925,71.94,77,0,8.81,LY,1635131793
171,port augusta,-32.5000,137.7667,77.70,16,0,2.19,AU,1635131796
374,saint-pierre,-21.3393,55.4781,75.56,73,0,3.44,RE,1635131983
536,sahuaripa,29.0500,-109.2333,71.91,47,0,4.32,MX,1635132136
556,tanda,26.5500,82.6500,75.94,61,0,6.06,IN,1635132154


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [23]:
# Store into variable named hotel_df
hotel_df = Ideal_Cities_df

In [24]:
# Add a "Hotel Name" column to the DataFrame.

# Base url to make the api call
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

# fixed Parameters
params = {
    "radius": 5000,
    "types": "lodging",
    "key": gkey
}

# Call API for each
for column, row in Ideal_Cities_df.iterrows(): 
    
    # Changing Parameters = coordinates
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f"{lat},{lng}"
    
    # Call
    response = requests.get(base_url, params=params).json()
    results = response['results']
    
    try:
        hotel_df.loc[column, 'Hotel Name'] = results[0]['name']
        
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
hotel_df

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date,Hotel Name
12,malangwa,26.8568,85.5581,76.64,71,0,1.43,NP,1635131648,Prem Meat House
70,tura,25.5198,90.2201,75.38,75,0,2.28,IN,1635131698,Hotel RIKMAN Continental
167,misratah,32.3754,15.0925,71.94,77,0,8.81,LY,1635131793,فندق الماسة
171,port augusta,-32.5000,137.7667,77.70,16,0,2.19,AU,1635131796,Majestic Oasis Apartments
374,saint-pierre,-21.3393,55.4781,75.56,73,0,3.44,RE,1635131983,Lindsey Hôtel
536,sahuaripa,29.0500,-109.2333,71.91,47,0,4.32,MX,1635132136,Estatua de Sahuaripa
556,tanda,26.5500,82.6500,75.94,61,0,6.06,IN,1635132154,MUMTAZ GANJ TANDA


In [25]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [26]:
# Add marker layer ontop of heat map
#markers = gmaps.marker_layer(locations)
hotel_layer = gmaps.symbol_layer(
    locations, fill_color='rgba(0, 150, 0, 0.4)',
    stroke_color='rgba(0, 0, 150, 0.4)', scale=2,
    info_box_content=hotel_info
)
# Display figure
fig.add_layer(markers)
fig.add_layer(hotel_layer)
fig


Figure(layout=FigureLayout(height='420px'))